# IT3103 - Assignment 1 - Advanced Topics in AI

## 🌼 Flower Classifier 🌼

### Team Members:
- Tan Qian Peng (234504H)
- Rethika
- Harith

### Introduction

This is a model that classifies flowers by different kinds (i.e. daisy, dandelion, roses, sunflowers, and tulips). In this notebook, we will be presenting our own neural network and how we will program our machine to understand and classify flowers into their own respective types.

The priority of our model would be to ensure that we have more than 70% accuracy, while avoiding as much overlapping as possible.

### Section 1: Data Preparation

Before we start this assignment, we must prepare the necessary dataset for this, so we will install necessary modules first.

In [25]:
import os
import tensorflow as tf
import keras
import numpy as np
import keras.layers as layers
from sklearn.metrics import classification_report

In [32]:
dataset_URL = "http://download.tensorflow.org/example_images/flower_photos.tgz"
path_to_zip = keras.utils.get_file('flower_photos.tgz', origin=dataset_URL, extract=True, cache_dir='.')
dataset_dir = os.path.join(os.path.dirname(path_to_zip), "flower_photos_extracted/flower_photos")
print(dataset_dir)

.\datasets\flower_photos_extracted/flower_photos


In [37]:
daisy_dir = os.path.join(dataset_dir, "daisy")
dandelion_dir = os.path.join(dataset_dir, "dandelion")
roses_dir = os.path.join(dataset_dir, "roses")
sunflowers_dir = os.path.join(dataset_dir, "sunflowers")
tulips_dir = os.path.join(dataset_dir, "tulips")

print('total daisy images:', len(os.listdir(daisy_dir)))
print('total dandelion images:', len(os.listdir(dandelion_dir)))
print('total roses images:', len(os.listdir(roses_dir)))
print('total sunflowers images:', len(os.listdir(sunflowers_dir)))
print('total tulips images:', len(os.listdir(tulips_dir)))

total daisy images: 633
total dandelion images: 898
total roses images: 641
total sunflowers images: 699
total tulips images: 799


# Splitting into training and validation sets

In [43]:
img_height, img_width = 128, 128
batch_size = 32

# resize all the images to the same size as expected by VGG model we downloaded above
image_size = (img_height, img_width)

train_ds = keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='int'
)
val_ds = keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='int'
)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [45]:
for images, labels in train_ds.take(1):
    print('images shape:', images.shape)
    print('labels shape:', labels.shape)
    print(tf.squeeze(labels))

images shape: (32, 128, 128, 3)
labels shape: (32,)
tf.Tensor([1 4 1 2 4 2 4 0 4 4 4 2 1 3 4 0 3 4 3 4 0 2 3 4 3 4 1 0 1 0 3 4], shape=(32,), dtype=int32)


In [49]:
## print out the class indices
print(train_ds.class_names)
print(val_ds.class_names)

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
